# Model Prediction Process
* Preprocess the test dataset
* Predict 
* Save results to csv

################################################ Imports & Constants ####################################

In [ ]:
import os
from config import config

CODA_DLL_PATH = config["preprocessing"]["coda_dll_path"]
H5_FILE = config["main"]["h5_file"]
CLASSES = config.get_classes()
IMG_SIZE = int(config["main"]["img_size"])

os.add_dll_directory(
    CODA_DLL_PATH
)  # https://github.com/tensorflow/tensorflow/issues/48868#issuecomment-841396124

import tensorflow as tf
import numpy as np

import vote
import results
import preprocessing


#####################################################################################################

In [ ]:
TEST_H5_FILE = "SynthText_test.h5"
X = preprocessing.load_unlabeled_dataset(TEST_H5_FILE)
X.head()

#### Normalize & prepare dataset

In [ ]:
x_norm = np.array(X["img"].to_list(), dtype=np.float32)/255

#### Load model & predict

In [ ]:
FINAL_MODEL = "final_model.h5"
model = tf.keras.models.load_model(FINAL_MODEL)
model.summary()

In [ ]:
# predict & fix prediction by votes
y_pred = model.predict(x_norm)

#### Fix predictions with voting & save to CSV

In [ ]:
y_pred_after_votes = vote.vote(X, y_pred)
# save to csv
results.to_csv(X, y_pred_after_votes, csv_file="results.csv")
